In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 690

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [4]:
def get_model_data(context_audio_features, audio_features):
    model_data = pd.DataFrame(columns=['context_audio_feature', 'audio_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
        audio_key = row["SCENE"] + "_u.wav"
        context_audio_key = row["SCENE"] + "_c.wav"
        model_data = model_data.append({'context_audio_feature': context_audio_features[context_audio_key],
                                    'audio_feature': audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [5]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data

In [6]:
class FNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'averaged_audio_feature']
            ctxt_features = self.data.loc[index, 'averaged_context_audio_feature']
            a=np.empty((1,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((ctxt_features, features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'averaged_audio_feature']
            ctxt_features = self.data.loc[index, 'averaged_context_audio_feature']
            final_features = np.hstack((ctxt_features, features))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)

class FNNNetSID(nn.Module):
    def __init__(self):
        super(FNNNetSID, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*desired_features*2, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*desired_features*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
class FNNNetSD(nn.Module):
    def __init__(self):
        super(FNNNetSD, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*(desired_features + desired_features + 1), hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*(desired_features + desired_features + 1))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
def predict_fnn(fnn_model, dataloader):
    prediction_list = []
    actual_list = []
    for data, target in dataloader:
        outputs = fnn_model(data)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
        actual_list.append(target)
    return prediction_list, actual_list

In [7]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')
with open('../../audio_features/feat_dict_context_librosa_lld.pickle', 'rb') as f:
    librosa_context_audio_features = pickle.load(f, encoding='latin1')
    
model_data = get_model_data(librosa_context_audio_features, librosa_audio_features)
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,context_audio_feature,audio_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-637.1869506835938, -527.0845538153164, -543...","[[-386.6164855957031, -649.6673512776692, -633...",0.0,NONE,SHELDON,25
1,"[[-625.8624267578125, -403.14568843397984, -38...","[[-255.5221405029297, -484.69307309105284, -52...",0.0,NONE,PENNY,15
2,"[[-500.3988952636719, -396.36312383413315, -22...","[[-569.0548095703125, -381.4147456242488, -221...",0.0,NONE,RAJ,21
3,"[[-313.1777038574219, -228.85994769251624, -35...","[[-237.61074829101562, -211.002773845897, -382...",1.0,PRO,HOWARD,7
4,"[[-337.88116455078125, -420.55431980762665, -3...","[[-530.5701293945312, -374.83951552370763, -42...",0.0,NONE,SHELDON,25


In [8]:
train_data, test_data = get_train_test_split(model_data, ['context_audio_feature', 'audio_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
fnn_train = train_data.copy()
fnn_test = test_data.copy()
fnn_train.reset_index(drop=True, inplace = True)
fnn_test.reset_index(drop=True, inplace = True)

        
fnn_train['averaged_audio_feature'] = fnn_train.loc[:, 'audio_feature']
fnn_train['averaged_context_audio_feature'] = fnn_train.loc[:, 'context_audio_feature']
for index, row in fnn_train.iterrows():
    audio = row['averaged_audio_feature']
    fnn_train.at[index, "averaged_audio_feature"] = np.array([np.mean(audio, axis=1)])
    context_audio = row['averaged_context_audio_feature']
    fnn_train.at[index, "averaged_context_audio_feature"] = np.array([np.mean(context_audio, axis=1)])

fnn_test['averaged_audio_feature'] = fnn_test.loc[:, 'audio_feature']
fnn_test['averaged_context_audio_feature'] = fnn_test.loc[:, 'context_audio_feature']
for index, row in fnn_test.iterrows():
    audio = row['averaged_audio_feature']
    fnn_test.at[index, "averaged_audio_feature"] = np.array([np.mean(audio, axis=1)])
    context_audio = row['averaged_context_audio_feature']
    fnn_test.at[index, "averaged_context_audio_feature"] = np.array([np.mean(context_audio, axis=1)])
    
fnn_train["sarcasm"] = fnn_train["sarcasm"].astype('int').to_numpy()
fnn_test["sarcasm"] = fnn_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 3) (961,) Test:  ((241, 3), (241,))


### Speaker InDependent

In [9]:
fnn_train_tensor = FNNTensorDataset(fnn_train[['averaged_context_audio_feature', 'averaged_audio_feature', 'sarcasm']], False)
fnn_test_tensor = FNNTensorDataset(fnn_test[['averaged_context_audio_feature', 'averaged_audio_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 100
valid_size = 0.2

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epochs = torch.utils.data.DataLoader(
    fnn_test_tensor, batch_size=fnn_test_tensor.__len__())

In [10]:
model = FNNNetSID()
print(model)

FNNNetSID(
  (fc1): Linear(in_features=1380, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(501):
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()*inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = predict_fnn(model, test_loader_epochs)
        print(classification_report(actuals[0].tolist(), predictions[0].tolist(), digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.4979    1.0000    0.6648       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4979       241
   macro avg     0.2490    0.5000    0.3324       241
weighted avg     0.2479    0.4979    0.3310       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.7857    0.2750    0.4074       120
           1     0.5628    0.9256    0.7000       121

    accuracy                         0.6017       241
   macro avg     0.6743    0.6003    0.5537       241
weighted avg     0.6738    0.6017    0.5543       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.5190    0.9083    0.6606       120
           1     0.6452    0.1653    0.2632       121

    accuracy                         0.5353       241
   macro avg     0.5821    0.5368    0.4619       241
weighted avg     0.5824    0.5353    0.4611  

Epoch: 500
              precision    recall  f1-score   support

           0     0.6694    0.6750    0.6722       120
           1     0.6750    0.6694    0.6722       121

    accuracy                         0.6722       241
   macro avg     0.6722    0.6722    0.6722       241
weighted avg     0.6722    0.6722    0.6722       241



### Speaker Dependent

In [15]:
fnn_train_tensor = FNNTensorDataset(fnn_train[['averaged_context_audio_feature', 'averaged_audio_feature', 'speaker_encode', 'sarcasm']], True)
fnn_test_tensor = FNNTensorDataset(fnn_test[['averaged_context_audio_feature', 'averaged_audio_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 100
valid_size = 0.2

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epochs = torch.utils.data.DataLoader(
    fnn_test_tensor, batch_size=fnn_test_tensor.__len__())

In [16]:
model = FNNNetSD()
print(model)

FNNNetSD(
  (fc1): Linear(in_features=1381, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(501):
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()*inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = predict_fnn(model, test_loader_epochs)
        print(classification_report(actuals[0].tolist(), predictions[0].tolist(), digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.4979    1.0000    0.6648       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4979       241
   macro avg     0.2490    0.5000    0.3324       241
weighted avg     0.2479    0.4979    0.3310       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5935    0.7667    0.6691       120
           1     0.6744    0.4793    0.5604       121

    accuracy                         0.6224       241
   macro avg     0.6340    0.6230    0.6147       241
weighted avg     0.6342    0.6224    0.6145       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.7711    0.5333    0.6305       120
           1     0.6456    0.8430    0.7312       121

    accuracy                         0.6888       241
   macro avg     0.7083    0.6882    0.6809       241
weighted avg     0.7081    0.6888    0.6811  

Epoch: 500
              precision    recall  f1-score   support

           0     0.6792    0.6000    0.6372       120
           1     0.6444    0.7190    0.6797       121

    accuracy                         0.6598       241
   macro avg     0.6618    0.6595    0.6584       241
weighted avg     0.6618    0.6598    0.6585       241

